# We Are How You Mean
## Important, real stats that matter

In [1]:
import pandas as pd
import csv
import os

## Managers and Team Names

In [2]:
# teams list
teams = {"Tim" : "Violent Tyreke Roadshow",
         "Scott" : "MT KilledbyLamaro",
         "Johnny" : "2 Brothers in a Chiefs Van",
         "JT" : "We Are Climbing High",
         "Matt" : "Mike Evans down your Vincents",
         "Tyler" : "Pirates of the Pancreas",
         "Brian" : "MuscularMoore'sTurbulentJuice",
         "Jason" : "footBall Fondlers",
         "Kevin" : "Three unmuscular Michaels jr.",
         "David" : "Ants In My Eyes Jackson"}

## Load and Update Scores and Matchups
### Functions for Scores

In [3]:
# Initialize weekly scores
def initialize_scores():
    weekly_scores = {manager: {f"Week {week}": 0 for week in range(1, 18)} for manager in teams}
    return weekly_scores

# Save scores to CSV
def save_scores_to_csv(weekly_scores, filename='weekly_scores.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ["Manager"] + [f"Week {week}" for week in range(1, 18)]
        writer.writerow(header)
        
        for manager, scores in weekly_scores.items():
            row = [manager] + [scores[f"Week {week}"] for week in range(1, 18)]
            writer.writerow(row)

# Load scores from CSV
def load_scores_from_csv(filename='weekly_scores.csv'):
    if not os.path.exists(filename):
        return initialize_scores()
    
    weekly_scores = {}
    with open(filename, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        for row in reader:
            manager = row[0]
            scores = {f"Week {i+1}": float(row[i+1]) if row[i+1] else 0 for i in range(17)}
            weekly_scores[manager] = scores
    return weekly_scores

# Update scores for a specific week
def update_weekly_scores(weekly_scores):
    # Ask for the week number
    while True:
        try:
            week = int(input("What week do you want to update? (1-17): "))
            if 1 <= week <= 17:
                break
            else:
                print("Please enter a valid week number between 1 and 17.")
        except ValueError:
            print("Please enter a valid number.")
    
    # Loop through each manager and update the score
    for manager in weekly_scores:
        try:
            score = float(input(f"What did {manager} score in Week {week}? "))
            weekly_scores[manager][f"Week {week}"] = score
        except ValueError:
            print("Please enter a valid score (numeric).")

    # Save updated scores to CSV
    save_scores_to_csv(weekly_scores)
    print(f"Scores for Week {week} updated successfully and saved.")

# Function to update an individual score
def update_individual_score(weekly_scores):
    # Ask for the manager's name
    manager = input("Which manager's score do you want to update? ").strip()
    
    if manager not in weekly_scores:
        print("Manager not found. Please make sure the name is correct.")
        return
    
    # Ask for the week number
    while True:
        try:
            week = int(input(f"What week do you want to update for {manager}? (1-17): "))
            if 1 <= week <= 17:
                break
            else:
                print("Please enter a valid week number between 1 and 17.")
        except ValueError:
            print("Please enter a valid number.")
    
    # Update the score
    try:
        score = float(input(f"What did {manager} score in Week {week}? "))
        weekly_scores[manager][f"Week {week}"] = score
        save_scores_to_csv(weekly_scores)
        print(f"Score for {manager} in Week {week} updated successfully and saved.")
    except ValueError:
        print("Please enter a valid score (numeric).")

### Functions for Matchups

In [4]:
# Initialize the weekly matchups
def initialize_matchups():
    weekly_matchups = {manager: {f"Week {week}": None for week in range(1, 18)} for manager in teams}
    return weekly_matchups

# Save matchups to CSV
def save_matchups_to_csv(weekly_matchups, filename='weekly_matchups.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        header = ["Manager"] + [f"Week {week}" for week in range(1, 18)]
        writer.writerow(header)
        
        for manager, matchups in weekly_matchups.items():
            row = [manager] + [matchups[f"Week {week}"] if matchups[f"Week {week}"] else "" for week in range(1, 18)]
            writer.writerow(row)

# Load matchups from CSV
def load_matchups_from_csv(filename='weekly_matchups.csv'):
    if not os.path.exists(filename):
        return initialize_matchups()
    
    weekly_matchups = {}
    with open(filename, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        for row in reader:
            manager = row[0]
            matchups = {f"Week {i+1}": row[i+1] if row[i+1] else None for i in range(17)}
            weekly_matchups[manager] = matchups
    return weekly_matchups

# Update matchups for a specific week
def update_weekly_matchups(weekly_matchups):
    # Ask for the week number
    while True:
        try:
            week = int(input("What week do you want to update matchups for? (1-17): "))
            if 1 <= week <= 17:
                break
            else:
                print("Please enter a valid week number between 1 and 17.")
        except ValueError:
            print("Please enter a valid number.")
    
    # Check if any matchups are already set for the chosen week
    already_set = any(weekly_matchups[manager][f"Week {week}"] is not None for manager in weekly_matchups)
    
    # Ask if user wants to reset if matchups already exist
    if already_set:
        reset = input(f"Matchups for Week {week} already exist. Do you want to reset them and start over? (yes/no): ").strip().lower()
        if reset != 'yes':
            print("No changes made. Exiting update process.")
            return

    # Reset all matchups for the specified week
    for manager in weekly_matchups:
        weekly_matchups[manager][f"Week {week}"] = None

    # Loop through each manager to set new matchups
    processed_managers = set()  # Keep track of managers we've already processed

    for manager in weekly_matchups:
        if manager in processed_managers:
            continue
        
        # Ask who this manager played
        while True:
            opponent = input(f"Who did {manager} play in Week {week}? ").strip()

            # Validate opponent exists, is not the same as the manager, and has not been processed yet
            if opponent not in teams or opponent == manager:
                print(f"{opponent} is not a recognized or valid opponent. Please try again.")
                continue
            
            # Check if opponent has already been processed
            if opponent in processed_managers:
                print(f"{opponent} has already been assigned a matchup. Please choose another opponent.")
                continue

            # If valid, update matchups symmetrically and mark both as processed
            weekly_matchups[manager][f"Week {week}"] = opponent
            weekly_matchups[opponent][f"Week {week}"] = manager
            processed_managers.update([manager, opponent])
            break

    # Save updated matchups to CSV
    save_matchups_to_csv(weekly_matchups)
    print(f"Matchups for Week {week} have been reset and updated successfully.")


# Function to update an individual matchup
def update_individual_matchup(weekly_matchups):
    # Ask for the manager's name
    manager = input("Which manager's matchup do you want to update? ").strip()
    
    if manager not in weekly_matchups:
        print("Manager not found. Please make sure the name is correct.")
        return
    
    # Ask for the week number
    while True:
        try:
            week = int(input(f"What week do you want to update for {manager}? (1-17): "))
            if 1 <= week <= 17:
                break
            else:
                print("Please enter a valid week number between 1 and 17.")
        except ValueError:
            print("Please enter a valid number.")
    
    # Ask for the opponent
    opponent = input(f"Who did {manager} play in Week {week}? ").strip()
    
    if opponent not in weekly_matchups:
        print(f"{opponent} is not a recognized manager. Please try again.")
        return
    
    # Update the matchup symmetrically
    weekly_matchups[manager][f"Week {week}"] = opponent
    weekly_matchups[opponent][f"Week {week}"] = manager
    save_matchups_to_csv(weekly_matchups)
    print(f"Matchup for {manager} in Week {week} updated successfully and saved.")

### Load existing stats

In [5]:
# Initialize or load the weekly scores
weekly_scores = load_scores_from_csv()

In [6]:
# Initialize or load the weekly matchups
weekly_matchups = load_matchups_from_csv()

### Update Scores

In [7]:
# # To update scores, call:
# update_weekly_scores(weekly_scores)

In [8]:
# # To correct or update and individual score
# update_individual_score(weekly_scores)

### Update Matchups

In [9]:
# # To update matchups for all managers in a specific week
# update_weekly_matchups(weekly_matchups)

In [10]:
# # To update an individual manager's matchup for a specific week
# update_individual_matchup(weekly_matchups)

## Basic statistics

#### Standings, Average, Average range, Best Score, Worst Score

## Alternate Records

#### Standings vs All Teams, Alternate Schedules, Standings w/ Median Scoring